# User-Defined Functions (UDFs)

### Objectives
1. Define a function
2. Create and apply a UDF
3. Register the UDF to use in SQL
4. Create and register a UDF with Python decorator syntax
5. Create and apply a Pandas (vectorized) UDF

### Methods
- [UDF Registration (`spark.udf`)](https://docs.databricks.com/spark/latest/spark-sql/udf-python.html): `register`
- [Built-In Functions](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/functions.html): `udf`
- [Python UDF Decorator](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.sql.functions.udf.html): `@udf`
- [Pandas UDF Decorator](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.sql.functions.pandas_udf.html): `@pandas_udf`


### Create inline sales dataset as CSV and load to Spark

In [ ]:
import pandas as pd

sales_data = [
    {"email": "a@gmail.com"},
    {"email": "b@yahoo.com"},
    {"email": "charlie@example.com"}
]

sales_pdf = pd.DataFrame(sales_data)
sales_path = "/dbfs/tmp/udf_sales_data.csv"
sales_pdf.to_csv(sales_path, index=False)

sales_df = (spark.read
            .option("header", "true")
            .option("inferSchema", "true")
            .csv(sales_path))

display(sales_df)

### Define a simple Python function

In [ ]:
def first_letter_function(email):
    return email[0]

first_letter_function("annagray@kaufman.com")

### Create and apply UDF

In [ ]:
from pyspark.sql.functions import udf, col

first_letter_udf = udf(first_letter_function)
display(sales_df.select(first_letter_udf(col("email")).alias("first_letter")))

### Register UDF to use in SQL

In [ ]:
sales_df.createOrReplaceTempView("sales")
first_letter_udf = spark.udf.register("sql_udf", first_letter_function)

In [ ]:
display(sales_df.select(first_letter_udf(col("email")).alias("first_letter")))

In [ ]:
# MAGIC %sql
SELECT sql_udf(email) AS first_letter FROM sales

### Define UDF using Python decorator syntax

In [ ]:
from pyspark.sql.functions import udf

@udf("string")
def first_letter_udf(email: str) -> str:
    return email[0]

In [ ]:
display(sales_df.select(first_letter_udf(col("email")).alias("first_letter")))

### Define and use Pandas (vectorized) UDF

In [ ]:
import pandas as pd
from pyspark.sql.functions import pandas_udf

@pandas_udf("string")
def vectorized_udf(email: pd.Series) -> pd.Series:
    return email.str[0]

display(sales_df.select(vectorized_udf(col("email")).alias("first_letter")))

### Register Pandas UDF to SQL namespace

In [ ]:
spark.udf.register("sql_vectorized_udf", vectorized_udf)

In [ ]:
# MAGIC %sql
SELECT sql_vectorized_udf(email) AS first_letter FROM sales